In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.api.types import CategoricalDtype
from statsmodels.tsa.seasonal import seasonal_decompose
import warnings
import statsmodels.api as sm
from pmdarima import auto_arima
from sklearn import metrics
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
warnings.filterwarnings("ignore")
from xgboost import XGBRegressor

color_pal = sns.color_palette()
plt.style.use('fivethirtyeight')

In [3]:
df = pd.read_excel('Liquidity-and-interbank-rate.xlsx',
                   sheet_name='LiquiditySeries',
                  skiprows=[0,1,3,4,5,2642]
                  )

In [4]:
df.head()

,Date (Nepali),Date (English),Balance at NRB minus CRR,Prime Rate,Secondary Rate,Overall Rate
0,2069-03-29,2012-07-13,73959.271,NaN,NaN,NaN
1,2069-04-05,2012-07-20,88835.736,NaN,NaN,NaN
2,2069-04-12,2012-07-27,82056.489,NaN,NaN,NaN
3,2069-04-26,2012-08-10,66216.424,NaN,NaN,NaN
4,2069-05-08,2012-08-24,50815.196,NaN,NaN,NaN


In [5]:
df.dtypes

Date (Nepali)                       object
Date (English)              datetime64[ns]
Balance at NRB minus CRR           float64
Prime Rate                          object
Secondary Rate                      object
Overall Rate                        object
dtype: object

In [6]:
df=df.drop(columns=['Date (Nepali)'])

In [7]:
df=df.rename(columns={"Date (English)":"Date"})

In [8]:
df.head()

,Date,Balance at NRB minus CRR,Prime Rate,Secondary Rate,Overall Rate
0,2012-07-13,73959.271,NaN,NaN,NaN
1,2012-07-20,88835.736,NaN,NaN,NaN
2,2012-07-27,82056.489,NaN,NaN,NaN
3,2012-08-10,66216.424,NaN,NaN,NaN
4,2012-08-24,50815.196,NaN,NaN,NaN


In [9]:
df.columns = df.columns.str.replace(' ', '_')

In [10]:
df.head()

,Date,Balance_at_NRB_minus_CRR,Prime_Rate,Secondary_Rate,Overall_Rate
0,2012-07-13,73959.271,NaN,NaN,NaN
1,2012-07-20,88835.736,NaN,NaN,NaN
2,2012-07-27,82056.489,NaN,NaN,NaN
3,2012-08-10,66216.424,NaN,NaN,NaN
4,2012-08-24,50815.196,NaN,NaN,NaN


In [11]:
df['Date']=pd.to_datetime(df['Date'])

In [12]:
df['Date']=df['Date'].dt.date

In [13]:
df['Date']=pd.to_datetime(df['Date'])

In [14]:
filtered_df = df[df['Date'] > '2016-08-09']

In [15]:
filtered_df=filtered_df.drop(columns=['Prime_Rate', 'Secondary_Rate','Overall_Rate'])

In [16]:
filtered_df.head()

,Date,Balance_at_NRB_minus_CRR
275,2016-08-10,36101.290972
276,2016-08-11,31244.397186
277,2016-08-13,29476.080942
278,2016-08-14,21602.674977
279,2016-08-15,24527.835281


In [17]:
filtered_df=filtered_df.rename(columns={"Date":"ds","Balance_at_NRB_minus_CRR":"y"})

In [18]:
filtered_df.head()

,ds,y
275,2016-08-10,36101.290972
276,2016-08-11,31244.397186
277,2016-08-13,29476.080942
278,2016-08-14,21602.674977
279,2016-08-15,24527.835281


In [21]:
train_size = int(len(df) * 0.8)
train, test = filtered_df[:train_size], filtered_df[train_size:]

# Create the feature matrix and target vector
X_train = train['y'].shift().dropna().to_frame()
y_train = train['y'].shift(-1).dropna()
X_test = test['y'].shift().dropna().to_frame()
y_test = test['y'].shift(-1).dropna()

In [22]:
# Train the XGBoost model
xgb = XGBRegressor(n_estimators=1000, learning_rate=0.01)
xgb.fit(X_train, y_train)

# Make predictions
y_pred = xgb.predict(X_test)

In [24]:
# Evaluate the model
mse = metrics.mean_squared_error(y_test, y_pred)
rmse = mse**0.5
mae = abs(y_test - y_pred).mean()

print(f'MSE: {mse:.4f}, RMSE: {rmse:.4f}, MAE: {mae:.4f}')

MSE: 77941500.6901, RMSE: 8828.4484, MAE: 6253.2413


In [25]:
def timeseries_evaluation_metrics_func(y_true, y_pred):
    def mean_absolute_percentage_error(y_true, y_pred):
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    print('Evaluation metric results:-')
    print(f'MSE is : {metrics.mean_squared_error(y_true, y_pred)}')
    print(f'MAE is : {metrics.mean_absolute_error(y_true, y_pred)}')
    print(f'RMSE is : {np.sqrt(metrics.mean_squared_error(y_true, y_pred))}')
    print(f'MAPE is : {mean_absolute_percentage_error(y_true,y_pred)}')
    print(f'R2 is : {metrics.r2_score(y_true, y_pred)}',end='\n\n')

In [26]:
timeseries_evaluation_metrics_func(y_test,y_pred)

Evaluation metric results:-
MSE is : 77941500.6901486
MAE is : 6253.24127540534
RMSE is : 8828.448373873442
MAPE is : 296.3817983999643
R2 is : 0.3328718298759624

